# Assignment 2: Data Cleaning
## Group 105
- Natasa Bolic (300241734)
- Brent Palmer (300193610)
## Imports

In [220]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Part 1: Clean Data Checker

## Introduction

Paragraph here

## Dataset Description

**Url:** https://www.kaggle.com/datasets/ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training <br>
**Name:** Cafe Sales - Dirty Data for Cleaning Training <br>
**Author:** Ahmed Mohamed <br>
**Purpose:** The dirty cafe sales dataset was fabricated to practice data cleaning, deliberately including missing data, inconsistencies, and errors. The Kaggle description specifies that the dataset "can be used to practice cleaning techniques, data wrangling, and feature engineering."<br>
**Shape:** There are 10,000 rows and 8 columns. (10000, 8)<br>
**Features:** 
- `Transaction ID` (categorical): A unique id assigned to each transaction.
- `Item` (categorical): The name of the purchased item.
- `Quantity` (numerical): The count of the purchased item.
- `Price Per Unit` (numerical): The price of one unit of the purchased item, measured in dollars.
- `Total Spent` (numerical): The total amount spent in the transaction, measured in dollars. (Quantity * Price Per Unit)
- `Payment Method` (categorical): The transaction's method of payment.
- `Location` (categorical): The location of the transaction.
- `Transaction Date` (numerical): The transaction date.

## Loading Dataset and Basic Exploration

In [2]:
# Read in the dataset from a public repository
url = "https://raw.githubusercontent.com/Natasa127/CSI4142-A2/main/dirty_cafe_sales.csv"
sales = pd.read_csv(url)
sales.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [3]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [4]:
sales.shape

(10000, 8)

## Data Checks

### 1) Data Type Errors

This test checks the data type of an attribute whose entries should be numerical (either an integer or a float).

**References:** <br>
Converting to numeric: https://pandas.pydata.org/docs/reference/api/pandas.to_numeric.html <br>
Setting the type: https://www.geeksforgeeks.org/python-pandas-dataframe-astype/ <br>
Selecting rows in one dataframe but not in another: https://discovery.cs.illinois.edu/guides/DataFrame-Row-Selection/dataframe-isin-selection/

In [5]:
# Parameters to be edited by the user
attributes = ['Quantity', 'Price Per Unit', 'Total Spent']
datatypes = ['int', 'float']

test_attribute = 'Quantity'
test_datatype = 'int'


In [6]:
# Error check
def type_filter(df, col, datatype):
    # Creates a copy so that the original dataset is not modified
    df_filtered = df.copy()

    # Converts numeric data to a numeric type and sets all other values to NaN
    df_filtered[col] = pd.to_numeric(df_filtered[col], errors='coerce')
    # Removes NaN values to leave only numerical values
    df_filtered = df_filtered.dropna(subset=[col]).copy()
    
    if datatype == 'int':
        # Takes only the integer values
        df_filtered = df_filtered[df_filtered[col] % 1 == 0].copy()

        # Converts the type to integer (as opposed to float)
        df_filtered[col] = df_filtered[col].astype(datatype)

    # Returns the filtered dataset
    return df_filtered

checked_sales = type_filter(sales, test_attribute, test_datatype)
checked_sales.info()


<class 'pandas.core.frame.DataFrame'>
Index: 9521 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    9521 non-null   object
 1   Item              9206 non-null   object
 2   Quantity          9521 non-null   int32 
 3   Price Per Unit    9349 non-null   object
 4   Total Spent       9353 non-null   object
 5   Payment Method    7074 non-null   object
 6   Location          6412 non-null   object
 7   Transaction Date  9371 non-null   object
dtypes: int32(1), object(7)
memory usage: 632.3+ KB


In [7]:
# Accesses entries with invalid datatypes for the given column
invalid_type = sales[~sales.index.isin(checked_sales.index)]
# Obtains number of invalid entries
print(len(invalid_type))
# Displays 5 invalid entries
invalid_type.head()

479


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
20,TXN_3522028,Smoothie,ERROR,4.0,20.0,Cash,In-store,2023-04-04
55,TXN_5522862,Cookie,ERROR,1.0,2.0,Credit Card,Takeaway,2023-03-19
57,TXN_2080895,Cake,UNKNOWN,3.0,3.0,Digital Wallet,In-store,2023-04-19
66,TXN_8501819,Juice,NaN,3.0,6.0,Cash,NaN,2023-03-30
117,TXN_2148617,Juice,ERROR,3.0,9.0,Digital Wallet,UNKNOWN,2023-01-10


Results:

There are 479 rows with a quantity that is not an integer. This seems to occur when the value is unknown, and replaced by a string such as 'UNKNOWN' or 'ERROR' instead. For example, see the two rows below:

<u>Transaction ID / Item / Quantity
Transaction ID	/ Item	/ Quantity 	/ Price Per Unit	Total Spent	/ Payment Method	/ Location	/ Transaction Date</u>

TXN_3522028	/ Smoothie	/ ERROR	/ 4.0	/ 20.0	/ Cash	/ In-store	/ 2023-04-04

TXN_5522862	/ Cookie	/ ERROR	/ 1.0	/ 2.0	/ Credit Card	/ Takeaway	/ 2023-03-19

We perform data type checks for the rest of the numerical attributes so that the columns have the correct datatype in subsequent checks.

In [8]:
# Filter by type for the remaining numerical attributes
checked_sales = type_filter(checked_sales, attributes[1], datatypes[1])
checked_sales = type_filter(checked_sales, attributes[2], datatypes[1])
checked_sales.info()


<class 'pandas.core.frame.DataFrame'>
Index: 8544 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    8544 non-null   object 
 1   Item              8261 non-null   object 
 2   Quantity          8544 non-null   int32  
 3   Price Per Unit    8544 non-null   float64
 4   Total Spent       8544 non-null   float64
 5   Payment Method    6354 non-null   object 
 6   Location          5762 non-null   object 
 7   Transaction Date  8415 non-null   object 
dtypes: float64(2), int32(1), object(5)
memory usage: 567.4+ KB


### 2) Range Errors

This test checks the range of a numerical variable, which consists of checking if the value of the variable is within the minimum and maximum acceptable values.

In [9]:
# Parameters to be edited by the user
attributes = ['Quantity', 'Price Per Unit', 'Total Spent']

test_attribute = 'Quantity'

minimum = 1

maximum = 5

In [10]:
# Error check

# Extract values that are either above the maximum acceptable value or below the minimum acceptable value
invalid_range = checked_sales[(checked_sales[test_attribute] > maximum) | (checked_sales[test_attribute] < minimum)]
invalid_range

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date


Results:

There are no values for quantity that are outside of the acceptable range.

TODO: maybe we should add invalid entries (like -1)

### 3) Format Errors

This test checks that dates are stored in the correct format, i.e. YYYY-MM-DD.


References:

Convert to datetime: https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html

In [11]:
# Parameters to be edited by the user
attribute = 'Transaction Date'

In [12]:
# Error Check

def format_filter(df, col):
    df_filtered = df.copy()
    
    # Sets all values that are not dates into NaT (not a time)
    df_filtered[col] = pd.to_datetime(df_filtered[col], yearfirst = True, errors='coerce')
    
    # Removes NaN values to leave only the values in the right format
    df_filtered = df_filtered.dropna(subset=[col]).copy()
    
    # Print the DataFrame to check the result
    return df_filtered

new_checked_sales = format_filter(checked_sales, attribute)
new_checked_sales.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,NaN,2023-03-31


In [13]:
# Accesses entries with invalid format for the given column
invalid_format = checked_sales[~checked_sales.index.isin(new_checked_sales.index)]
# Obtains number of invalid entries
print(len(invalid_format))
# Displays 5 invalid entries
invalid_format.head()

385


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
11,TXN_3051279,Sandwich,2,4.0,8.0,Credit Card,Takeaway,ERROR
29,TXN_7640952,Cake,4,3.0,12.0,Digital Wallet,Takeaway,ERROR
33,TXN_7710508,UNKNOWN,5,1.0,5.0,Cash,NaN,ERROR
77,TXN_2091733,Salad,1,5.0,5.0,NaN,In-store,NaN
103,TXN_7028009,Cake,4,3.0,12.0,NaN,Takeaway,ERROR


Results:

There are 385 rows where the transaction date is in the wrong format. For example, see the two rows below:

<u>Transaction ID / Item / Quantity
Transaction ID	/ Item	/ Quantity 	/ Price Per Unit	Total Spent	/ Payment Method	/ Location	/ Transaction Date</u>

TXN_3051279	/ Sandwich	/ 2	/ 4.0	/ 8.0	/ Credit Card	/ Takeaway	/ ERROR

TXN_7640952	/ Cake	/ 4	/ 3.0	/ 12.0	/ Digital Wallet	/ Takeaway	/ ERROR

# Part 2: Imputation

## Introduction

Paragraph here

## Dataset Description

**Url:** https://www.kaggle.com/datasets/uciml/autompg-dataset <br>
**Name:** Auto-mpg Dataset <br>
**Author:** UCI Machine Learning Repository (originally from StatLib library, maintained at Carnegie Mellon University) <br>
**Purpose:** The dataset includes the technical specifications of cars. The original purpose of collection is not explicitly listed on Kaggle, but it appears to have been collected to evaluate how fuel consumption relates to various other attributes of vehicles (e.g., horsepower, weight). In 1993, Ross Quinlan used the dataset to train a machine learning model to predict fuel consumption based on the other eight features.<br>
**Shape:** There are 398 rows and 9 columns. (398, 9)<br>
**Features:** Further explanation of the features retrieved from https://code.datasciencedojo.com/tshrivas/dojoHub/tree/master/Auto%20MPG%20Data%20Set
- `mpg` (numerical): The vehicle's fuel efficiency, measured in miles per gallon (mpg).
- `cylinders` (categorical): The number of cylinders in the vehicle's engine.
- `displacement` (numerical): The total volume of the cylinders in the vehicle, measured in cubic inches.
- `horsepower` (numerical): A measurement of the vehicle's engine's power.
- `weight` (numerical): The weight of the vehicle, measured in pounds (lbs).
- `acceleration` (numerical): Time to go from 0 to 60 miles per hour, measured in seconds.
- `model year` (categorical): The year of release of the vehicle.
- `origin` (categorical): The region of manufacturing.
    - 1: USA
    - 2: Europe
    - 3: Japan
- `car name` (categorical): The name of the vehicle.

**Missing Values:** Yes, there are missing values. In particular, horsepower has 6 missing values.

## Loading Dataset and Basic Exploration

In [14]:
# Read in the dataset from a public repository
url = "https://raw.githubusercontent.com/Natasa127/CSI4142-A2/refs/heads/main/auto-mpg.csv"
auto_df = pd.read_csv(url)
auto_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [15]:
auto_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   car name      398 non-null    object 
dtypes: float64(3), int64(4), object(2)
memory usage: 28.1+ KB


In [16]:
auto_df.shape

(398, 9)

### Basic Cleaning
Since there are only six missing values in the entire dataset (1.5% of the rows), we decided to use the listwise removal approach to delete these rows. The assignment requires simulating missing values (as indicated by part b), which enables us to do cross validation since we know the true values of the data. Thus, deleting these six rows in advance will make the cross-validation approach possible. Since it is such a small amount of data, this deletion will not significantly impact our analysis.

**References:** <br>
Listwise removal: https://saturncloud.io/blog/how-to-remove-rows-with-specific-values-in-pandas-dataframe/ <br>
Series equality: https://pandas.pydata.org/docs/reference/api/pandas.Series.eq.html

In [17]:
auto_df = auto_df.drop(auto_df[auto_df['horsepower'] == '?'].index)

In [19]:
auto_df['horsepower'].eq('?').sum()

0

The six rows have successfully been removed, and we are now ready for imputation and evaluation using cross-validation with simulation.

## Imputation Tests

### Imputation Test 1: Random Sample Imputation on Acceleration (Univariate)

#### (a) Chosen Attribute

We have chosen to test random sample imputation (univariate) on the acceleration attribute.

#### (b) Simulate Missing Values

We are simulating missing acceleration values using the MCAR approach, where the missing acceleration values are chosen completely at random regardless of their own value or the values of the other attributes.

We are simulating the missing values in a copy of the original DataFrame such that we can evaluate the imputation accuracy afterwards by comparing the imputed acceleration values with the true acceleration values.

**References:** <br>
MCAR: https://www.kaggle.com/code/yassirarezki/handling-missing-data-mcar-mar-and-mnar-part-i <br>
Loc Documentation: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html <br>
Copy Documentation: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.copy.html <br>
Fixing Random Seed: https://stackoverflow.com/questions/21494489/what-does-numpy-random-seed0-do

In [217]:
# Create a copy of the DataFrame
missing_acceleration_df = auto_df.copy()

# Set the random seed to make the results reproducible (comment this out to try on truly random missing values)
np.random.seed(0)

# Generate a series that holds True for rows that will be missing, and False for rows that will persist
missing_percent = 0.1
missing_values = np.random.choice([True, False], len(auto_df), p=[missing_percent, 1 - missing_percent])

# Replace the acceleration with NaN where the missing_values series is True
missing_acceleration_df.loc[missing_values, "acceleration"] = np.nan

# Check to see if values are missing
missing_acceleration_df['acceleration'].isna().sum()

39

#### (c) Program an Imputation Approach to Replace the Missing Values

We have chosen random sample imputation as our first imputation approach. Random sample imputation is a univariate technique.

**References:** <br>
Sample Documentation: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html

In [218]:
# Save a series of the non-missing acceleration values
acceleration_values = missing_acceleration_df.loc[missing_acceleration_df['acceleration'].notna(), 'acceleration']

# Sample a number of non-missing acceleration values with replacement equal to the number of missing values
# Note that random state is set to one to make the results reproducible. This can be removed for truly random samples
sampled_acceleration_values = acceleration_values.sample(missing_acceleration_df['acceleration'].isna().sum(), replace=True, random_state=1).values

# Replace the missing values with sampled acceleration values
missing_acceleration_df.loc[missing_values, "acceleration"] = sampled_acceleration_values

# Check to see if values are missing
missing_acceleration_df['acceleration'].isna().sum()

0

#### (d) Evaluate to What Extent Your Approach is Finding the Missing Values

Since we are using the cross-validation with simulation approach where we simulate missing data, we can compare our imputed values to the true values.

##### Evaluation Using MSE

We will first use the Mean Squared Error (MSE) approach, where we take the average of the squared difference of each imputed value with the respective true value.

**References:** <br>
Mean Squared Error: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html

In [219]:
# First determine the original values of the missing data
original_values = auto_df.loc[missing_values, 'acceleration'].values

# Then determine the imputed values of the missing data
imputed_values = missing_acceleration_df.loc[missing_values, 'acceleration'].values

# Compute MSE
mse = mean_squared_error(original_values, imputed_values)
mse

9.729999999999997

Thus, the mean square error is 9.73 using random sample imputation. The MSE is not particularly intuitive to interpret as a standalone value since it is measured in squared units. A great way to interpret MSE is to compare it to the MSE of a baseline method, like median imputation. In the following cell we will perform median imputation, and compare the MSE values to evaluate the effectiveness of the random sampling method.

In [215]:
# Replace the imputed values with NaN where the missing_values series is True.
missing_acceleration_df.loc[missing_values, "acceleration"] = np.nan

# Save the median of the non-missing acceleration values
acceleration_median = missing_acceleration_df.loc[missing_acceleration_df['acceleration'].notna(), 'acceleration'].median()

# Replace the missing values with median
missing_acceleration_df.loc[missing_values, "acceleration"] = acceleration_median

# Compute MSE
original_values = auto_df.loc[missing_values, 'acceleration'].values
imputed_values = missing_acceleration_df.loc[missing_values, 'acceleration'].values
median_mse = mean_squared_error(original_values, imputed_values)
median_mse

5.330512820512821

Given that random sampling gave an MSE of 9.73 and default value imputation using the median gave an MSE of 5.33, the random sampling approach is not very good comparatively in this case. 

##### Evaluation Using MAE

A second approach to evaluation is Mean Absolute Error (MAE), which is more intuitive since the units of MAE are the same as the original data. MAE takes the average absolute difference between each imputed value with the respective true value.

**References:** <br>
Mean Squared Error: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html

In [228]:
mae = mean_absolute_error(original_values, imputed_values)
mae

2.6128205128205124

The MAE is 2.61, which means that on average, the imputed acceleration had an error of 2.61 seconds. In the context of a vehicle's acceleration in seconds, this is not that bad.

##### Evaluation Conclusion
Despite the random sampling MSE being notably worse than the default value imputation using the median MSE, random sampling is still usable as seen through the reasonable MAE score of 2.61 seconds. 